In [1]:
import pandas as pd
import numpy as np
import glob
import re
import os
import warnings
import string
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', 400)

In [2]:
file_prefix='done_bug_'
file_pattern = os.path.join("./data/jira_trem/", f'{file_prefix}*.csv')
csv_files = glob.glob(file_pattern)

dataframes = []
for file in csv_files:
    dataframes.append(pd.read_csv(file))
df = pd.concat(dataframes, ignore_index=True)
df.head()

,Summary,Issue key,Issue id,Issue Type,Status,Project key,Project name,Project type,Project lead,Project lead id,Project description,Project url,Priority,Resolution,Assignee,Assignee Id,Reporter,Reporter Id,Creator,Creator Id,Created,Updated,Last Viewed,Resolved,Affects versions,Affects versions.1,Affects versions.2,Affects versions.3,Fix versions,Fix versions.1,Fix versions.2,Due date,Votes,Labels,Labels.1,Labels.2,Description,Environment,Watchers,Watchers.1,Watchers.2,Watchers.3,Watchers.4,Watchers.5,Watchers.6,Watchers.7,Watchers.8,Watchers Id,Watchers Id.1,Watchers Id.2,Watchers Id.3,Watchers Id.4,Watchers Id.5,Watchers Id.6,Watchers Id.7,Watchers Id.8,Log Work,Log Work.1,Log Work.2,Log Work.3,Log Work.4,Log Work.5,Log Work.6,Log Work.7,Log Work.8,Log Work.9,Original estimate,Remaining Estimate,Time Spent,Work Ratio,Σ Original Estimate,Σ Remaining Estimate,Σ Time Spent,Security Level,Inward issue link (Blocks),Outward issue link (Blocks),Inward issue link (Cloners),Outward issue link (Cloners),Inward issue link (Defect),Outward issue link (Defect),Inward issue link (Duplicate),Inward issue link (Polaris datapoint issue link),Inward issue link (Problem/Incident),Outward issue link (Problem/Incident),Inward issue link (Relates),Inward issue link (Relates).1,Inward issue link (Relates).2,Outward issue link (Relates),Outward issue link (Relates).1,Outward issue link (Relates).2,Attachment,Attachment.1,Attachment.2,Attachment.3,Attachment.4,Attachment.5,Attachment.6,Attachment.7,Attachment.8,Attachment.9,Attachment.10,Attachment.11,Attachment.12,Attachment.13,Attachment.14,Attachment.15,Attachment.16,Attachment.17,Attachment.18,Attachment.19,Attachment.20,Custom field (Actual end),Custom field (Actual start),Custom field (Affected hardware),Custom field (Affected services),Custom field (Approvals),Custom field (Backout plan),Custom field (Baseline end date),Custom field (Baseline start date),Custom field (Billable),Custom field (Category),Custom field (Change reason),Custom field (Change risk),Custom field (Change type),Custom field (Checklists),Custom field (Client),Custom field (Craft),Custom field (Date of First Response),Custom field (Date of first comment),Custom field (Dev Estimation (mds)),Custom field (Development),Custom field (End date),Custom field (Engagement Type),Custom field (Epic Color),Custom field (Epic Link),Epic Link Summary,Custom field (Epic Name),Custom field (Epic Status),Custom field (Estimation Zoom Level),Custom field (FirstBundle),Custom field (Flagged),Custom field (Goals),Custom field (Impact),Custom field (Implementation plan),Custom field (Investigation reason),Custom field (Issue Origin),Custom field (Issue color),Custom field (Locked forms),Custom field (Major incident),Custom field (Open forms),Custom field (Operational categorization),Custom field (Pending reason),Custom field (Percentage of allocation in the project),Custom field (Planned end),Custom field (Planned start),Custom field (Product categorization),Custom field (Program Increment),Custom field (Project overview key),Custom field (Project overview status),Custom field (QA Automation Estimation (mds)),Custom field (QA Manual Estimation (mds)),Custom field (Rank),Custom field (Reopen Counter),Custom field (Request Type),Custom field (Request language),Custom field (Request participants),Custom field (Responders),Custom field (Root cause),Satisfaction rating,Custom field (Satisfaction date),Custom field (Scope),Custom field (Sentiment),Custom field (Severity),Custom field (Severity).1,Custom field (Source),Sprint,Sprint.1,Sprint.2,Sprint.3,Sprint.4,Sprint.5,Sprint.6,Sprint.7,Sprint.8,Sprint.9,Sprint.10,Custom field (Start date),Custom field (Story Points),Custom field (Story point estimate),Custom field (Submitted forms),Custom field (TEST Long List),Custom field (TTFR),Custom field (TTR),Custom field (Target end),Custom field (Target start),Custom field (Task progress),Custom field (Team),Custom field (Test Environment),Custom field 

In [3]:
df.shape

(2630, 365)

In [4]:
text_df = df[['Summary','Description','Priority']]
text_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2630 entries, 0 to 2629
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Summary      2630 non-null   object
 1   Description  2578 non-null   object
 2   Priority     2630 non-null   object
dtypes: object(3)
memory usage: 61.8+ KB


In [5]:
text_df['Description'] = text_df['Description'].fillna('')

In [6]:
import string, re, nltk
import spacy
from string import punctuation
from nltk.tokenize import word_tokenize, RegexpTokenizer
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer

In [7]:
#定义文本处理函数
# RegexpTokenizer
regexp = RegexpTokenizer("[\w']+")

# 转换为小写
def convert_to_lowercase(text):
    return text.lower()

# 去除文本两边空格
def strip_text(text):
    return text.strip()

# 移除标点符号
def remove_punctuation(text):
    punct_str = string.punctuation
    punct_str = punct_str.replace("'", "")
    return re.sub(f"[{re.escape(punct_str)}]", " ", text)

# 移除数字token
def remove_number_token(text):
    words = text.split()
    # 过滤掉纯数字的词
    filtered_words = [word for word in words if not re.match(r'^\d+$', word)]
    # 将词按空格合并成句子
    combined_sentence = ' '.join(filtered_words).strip()
    return combined_sentence

# 移除html标签
def remove_html(text):
    html = re.compile(r'<.*?>')
    return html.sub(' ', text)

# 移除表情
def remove_emoji(text):
    emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           "]+", flags = re.UNICODE)
    return emoji_pattern.sub(' ', text)

# 移除http链接
def remove_http(text):
    http = "https?://\S+|www\.\S+" # matching strings beginning with http (but not just "http")
    pattern = r"({})".format(http) # creating pattern
    return re.sub(pattern, " ", text)

# Dictionary of acronyms
acronyms_url = './data/english_acronyms.json'
acronyms_dict = pd.read_json(acronyms_url, typ = 'series')
acronyms_list = list(acronyms_dict.keys())

# convert contractions in a text
def convert_acronyms(text):
    words = []
    for word in regexp.tokenize(text):
        if word in acronyms_list:
            words = words + acronyms_dict[word].split()
        else:
            words = words + word.split()
    
    text_converted = " ".join(words)
    return text_converted

# Dictionary of contractions
contractions_url = './data/english_contractions.json'
contractions_dict = pd.read_json(contractions_url, typ = 'series')
# List of contractions
contractions_list = list(contractions_dict.keys())

# convert contractions in a text
def convert_contractions(text):
    words = []
    for word in regexp.tokenize(text):
        if word in contractions_list:
            words = words + contractions_dict[word].split()
        else:
            words = words + word.split()
    
    text_converted = " ".join(words)
    return text_converted

# 移除文本中包含的image tag
def remove_image_tags(text):
    # Define the regular expression pattern to match the image tags
    pattern = re.compile(r'!.*?!')
    cleaned_text = pattern.sub(' ', text)
    return cleaned_text

# 移除停用词
def remove_stopwords(text):
    stop_words = set(stopwords.words('english'))
    
    # 使用正则表达式 tokenizer 处理缩写和标点
    tokenizer = RegexpTokenizer(r'\w+\'?\w+|\w+')
    words = tokenizer.tokenize(text)
    
    filtered_words = [word for word in words if word.lower() not in stop_words]
    return ' '.join(filtered_words)

# Stemming 词干提取，stemming. 如 "running", "runner" 会被转换成 "run".
stemmer = PorterStemmer()
def text_stemmer(text):
    text_stem = " ".join([stemmer.stem(word) for word in regexp.tokenize(text)])
    return text_stem

# 移除非字母的词
def discard_non_alpha(text):
    word_list_non_alpha = [word for word in regexp.tokenize(text) if word.isalpha()]
    text_non_alpha = " ".join(word_list_non_alpha)
    return text_non_alpha

def text_lemmatizer(text):
    words = text.split()
    lemmatizer = WordNetLemmatizer()
    words = [lemmatizer.lemmatize(word) for word in words]
    return ' '.join(words)

def preprocess_text(text):
    text = remove_html(text)
    text = remove_http(text)
    text = remove_emoji(text)
    text = remove_image_tags(text)
    text = convert_to_lowercase(text)
    text = re.sub('\n', ' ', text) # converting text to one line
    text = strip_text(text)
    text = remove_punctuation(text)
    text = convert_acronyms(text)
    text = convert_contractions(text)
    text = remove_stopwords(text)
    text = discard_non_alpha(text)
    text = text_lemmatizer(text)
    text = re.sub(' +', ' ', text)  # replace multiple spaces with a single space
    text = strip_text(text)
    return text

In [8]:
# 应用文本清洗函数
text_df['cleaned_summary'] = text_df['Summary'].apply(preprocess_text)
text_df['cleaned_description'] = text_df['Description'].apply(preprocess_text)

# 合并summary和description
text_df['text'] = text_df['cleaned_summary'] + ' ' + text_df['cleaned_description']

# 定义优先级映射
priority_mapping = {
    'Lowest': 0,
    'Low': 1,
    'Medium': 2,
    'High': 3,
    'Highest': 4
}

# 将优先级转换为数值标签
text_df['Priority'] = text_df['Priority'].map(priority_mapping)


In [9]:
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

In [11]:
MAX_NUM_WORDS = 5000
MAX_SEQUENCE_LENGTH = 100

tokenizer = Tokenizer(num_words=MAX_NUM_WORDS)
tokenizer.fit_on_texts(text_df['text'])
word_index = tokenizer.word_index

sequences = tokenizer.texts_to_sequences(text_df['text'])
X = pad_sequences(sequences, maxlen=MAX_SEQUENCE_LENGTH)
y = text_df['Priority'].values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

smote = SMOTE(random_state=42)
X_train_res, y_train_res = smote.fit_resample(X_train, y_train)

In [12]:
from keras.models import Sequential
from keras.layers import Embedding, SpatialDropout1D, LSTM, Dense, Dropout
from keras.optimizers import Adam
from scikeras.wrappers import KerasClassifier
from sklearn.model_selection import RandomizedSearchCV

In [13]:
# 定义LSTM+MLP模型构建函数
def create_model(embedding_dim=128, lstm_units=100, dense_units=128, dropout_rate=0.2, recurrent_dropout_rate=0.2, optimizer='adam'):
    model = Sequential()
    model.add(Embedding(input_dim=MAX_NUM_WORDS, output_dim=embedding_dim, input_length=MAX_SEQUENCE_LENGTH))
    model.add(SpatialDropout1D(dropout_rate))
    model.add(LSTM(lstm_units, dropout=dropout_rate, recurrent_dropout=recurrent_dropout_rate))
    model.add(Dense(dense_units, activation='relu'))
    model.add(Dropout(dropout_rate))
    model.add(Dense(len(set(y)), activation='softmax'))
    model.compile(loss='sparse_categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    return model

# 使用KerasClassifier包装模型以便于随机搜索
model = KerasClassifier(
    build_fn=create_model,
    verbose=0
)

In [14]:
# 定义超参数搜索空间
param_dist = {
    'model__embedding_dim': [64, 128, 256],
    'model__lstm_units': [50, 100, 150],
    'model__dense_units': [64, 128, 256],
    'model__dropout_rate': [0.2, 0.3, 0.4],
    'model__recurrent_dropout_rate': [0.2, 0.3, 0.4],
    'model__optimizer': ['adam', 'rmsprop'],
    'epochs': [5, 10, 15],
    'batch_size': [32, 64, 128]
}

# 使用随机搜索进行超参数优化
random_search = RandomizedSearchCV(estimator=model, param_distributions=param_dist, n_iter=20, cv=3, random_state=42, n_jobs=-1)
random_search.fit(X_train_res, y_train_res)

# 打印最佳超参数
print("Best Parameters:", random_search.best_params_)

/opt/anaconda3/envs/ds/lib/python3.8/site-packages/scikeras/wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
/opt/anaconda3/envs/ds/lib/python3.8/site-packages/scikeras/wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
/opt/anaconda3/envs/ds/lib/python3.8/site-packages/scikeras/wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
/opt/anaconda3/envs/ds/lib/python3.8/site-packages/scikeras/wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
/opt

Best Parameters: {'model__recurrent_dropout_rate': 0.3, 'model__optimizer': 'rmsprop', 'model__lstm_units': 150, 'model__embedding_dim': 128, 'model__dropout_rate': 0.4, 'model__dense_units': 256, 'epochs': 5, 'batch_size': 32}


In [15]:
# 使用最佳模型进行预测
best_model = random_search.best_estimator_
y_pred_lstm = best_model.predict(X_test)

In [19]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import TfidfVectorizer

In [20]:
# 用随机森林作为基准参照
vectorizer = TfidfVectorizer(max_features=5000)
X_tfidf = vectorizer.fit_transform(text_df['text'])

X_train_tfidf, X_test_tfidf, y_train_tfidf, y_test_tfidf = train_test_split(X_tfidf, y, test_size=0.2, random_state=42, stratify=y)

X_train_res_tfidf, y_train_res_tfidf = smote.fit_resample(X_train_tfidf, y_train_tfidf)

rf = RandomForestClassifier()

param_dist_rf = {
    'n_estimators': [100, 200, 300],
    'max_depth': [10, 20, 30, None],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'bootstrap': [True, False]
}

random_search_rf = RandomizedSearchCV(rf, param_distributions=param_dist_rf, n_iter=50, cv=3, random_state=42, n_jobs=-1)
random_search_rf.fit(X_train_res_tfidf, y_train_res_tfidf)

best_rf = random_search_rf.best_estimator_
y_pred_rf = best_rf.predict(X_test_tfidf)


In [16]:
from sklearn.metrics import classification_report, confusion_matrix

In [17]:
# LSTM+MLP模型评估
print("LSTM+MLP Model Performance:")
print(classification_report(y_test, y_pred_lstm))
print(confusion_matrix(y_test, y_pred_lstm))

LSTM+MLP Model Performance:
              precision    recall  f1-score   support

           0       0.11      0.18      0.14        11
           1       0.37      0.08      0.13        88
           2       0.61      0.64      0.62       299
           3       0.25      0.44      0.32        85
           4       0.14      0.09      0.11        43

    accuracy                           0.46       526
   macro avg       0.30      0.29      0.26       526
weighted avg       0.46      0.46      0.44       526

[[  2   1   6   0   2]
 [  3   7  62  10   6]
 [ 11  10 191  73  14]
 [  1   1  44  37   2]
 [  1   0  12  26   4]]


In [21]:
# 随机森林基线模型评估
print("Random Forest Baseline Model Performance:")
print(classification_report(y_test_tfidf, y_pred_rf))
print(confusion_matrix(y_test_tfidf, y_pred_rf))

Random Forest Baseline Model Performance:
              precision    recall  f1-score   support

           0       0.00      0.00      0.00        11
           1       0.22      0.06      0.09        88
           2       0.59      0.91      0.71       299
           3       0.41      0.13      0.20        85
           4       0.50      0.14      0.22        43

    accuracy                           0.56       526
   macro avg       0.34      0.25      0.24       526
weighted avg       0.48      0.56      0.47       526

[[  0   3   8   0   0]
 [  0   5  82   1   0]
 [  0  15 272   9   3]
 [  0   0  71  11   3]
 [  0   0  31   6   6]]
